작성일자: 2022-08-04<br>
작성자: 정예준

기존 방식: 자식 클래스에서 부모 클래스의 ```__init__``` 메서드를 호출하는 방법으로 부모 클래스를 초기화

In [1]:
class MyBaseClass(object):
    def __init__(self, value):
        self.value = value
    
class MyChildClass(MyBaseClass):
    def __init__(self):
        MyBaseClass.__init__(self, 5)

직접 슈퍼클래스의 ```__init__```을 호출하는 행위는 문제를 발생시킴<br>
class문의 argument 순서대로 불러오는 것을 원하는데 그렇지 못한 것을 볼 수 있음<br>

In [2]:
class TimesTwo(object):
    def __init__(self):
        self.value *= 2
    
class PlusFive(object):
    def __init__(self):
        self.value += 5

애초에 ```__init__```아래에 호출 순서를 똑같이 적어 놓았는데 당연히 똑같은 결과가 나오는 거 아닌가 싶음<br>
클래스를 사용할 떄 arguments의 순서대로 호출하는 것을 원하는 것 같다

In [3]:
class OneWay(MyBaseClass, TimesTwo, PlusFive):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        TimesTwo.__init__(self)
        PlusFive.__init__(self)

foo = OneWay(5)
print(foo.value)

class AnotherWay(MyBaseClass, PlusFive, TimesTwo):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        TimesTwo.__init__(self)
        PlusFive.__init__(self)
        
bar = AnotherWay(5)
print(bar.value)

15
15


또 다른 문제 : 다이아몬드 상속<br>
서브클래스가 계층 구조에서 같은 슈퍼클래스를 둔 서로 다른 두 클래스에서 상속받을때 발생<br>
공통 슈퍼클래스의 ```__init__```메서드를 여러 번 실행하게 해서 예상치 못한 동작을 일으킴<br>
MyBaseClass를 상속받은 자식 클래스 2개를 정의

In [7]:
class TimesFive(MyBaseClass):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        self.value *= 5
    
class PlusTwo(MyBaseClass):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        self.value += 2

두 클래스 모두를 상속받는 자식 클래스를 정의하여 MyBaseClass를 다이아몬드 꼭대기로 만듬

In [8]:
class ThisWay(TimesFive, PlusTwo):
    def __init__(self, value):
        TimesFive.__init__(self, value)
        PlusTwo.__init__(self, value)

In [9]:
foo = ThisWay(5)
print(foo.value)

7


### 파이썬 2.2에서의 해결방법: 내장함수 ```super```와 ```MRO(매서드 해석 순서, Method Resolution Order)```<br>
MRO는 어떤 슈퍼클래스로부터 초기화하는지를 정하고 다이아몬드 계층 구조에 있는 공통 슈퍼클래스를 단 한 번만 실행하게 함<br>
같은 다이아몬드 클래스를 super를 이용해 부모 클래스를 초기화

In [10]:
# 파이썬 2
class TimesFiveCorrect(MyBaseClass):
    def __init__(self, value):
        super(TimesFiveCorrect, self).__init__(value)
        self.value *= 5
        
class PlusTwoCorrect(MyBaseClass):
    def __init__(self, value):
        super(PlusTwoCorrect, self).__init__(value)
        self.value += 2

다이아몬드의 꼭대기인 ```MyBaseClass.__init__```이 한번만 실행됨<br>
다른 부모 클래스는 class문으로 지정한 순서대로 실행됨

In [12]:
# 파이썬 2
class GoodWay(TimesFiveCorrect, PlusTwoCorrect):
    def __init__(self, value):
        super(GoodWay, self).__init__(value)

foo = GoodWay(5)
print(foo.value)

35


보아하니 class 문의 argument 역순으로 실행하는 듯 함<br>
순서를 확인해보기 위해 mro 정의 순서를 보자

In [15]:
from pprint import pprint
pprint(GoodWay.mro())

[<class '__main__.GoodWay'>,
 <class '__main__.TimesFiveCorrect'>,
 <class '__main__.PlusTwoCorrect'>,
 <class '__main__.MyBaseClass'>,
 <class 'object'>]


In [18]:
# 파이썬 2
class GoodWay1(PlusTwoCorrect, TimesFiveCorrect):
    def __init__(self, value):
        super(GoodWay1, self).__init__(value)

foo = GoodWay1(5)
print(foo.value)

pprint(GoodWay1.mro())

27
[<class '__main__.GoodWay1'>,
 <class '__main__.PlusTwoCorrect'>,
 <class '__main__.TimesFiveCorrect'>,
 <class '__main__.MyBaseClass'>,
 <class 'object'>]


class문의 argument 역순으로 실행하는 것을 알 수 있음<br>
파이썬 3에서는 super를 호출할 때 클래스의 이름을 지정하지 않아도 됨(```__class__```와 self를 인수로 넘기기 때문)<br>
super는 명확하고 간결하며 항상 제대로 동작하기 떄문에 언제나 사용해야 함

In [19]:
class Explicit(MyBaseClass):
    def __init__(self, value):
        super(Explicit, self).__init__(value * 2)
        
class Implicit(MyBaseClass):
    def __init__(self, value):
        super().__init__(value * 2)
        
print(Explicit(10).value == Implicit(10).value)

True
